# Elope

In [23]:
import pandas as pd
import cv2
import numpy as np
from keras.models import model_from_json
import tensorflow as tf

In [24]:
def load_generators(img_size = 448):
    # This function loads the generator for the test.
    # This first approach takes the test images from the disk
    
    from keras.preprocessing.image import ImageDataGenerator

    img_size = 448
    test_dir = 'C:/Users/julen/OneDrive/Escritorio/IA/CS577-Deep-Learning/Project/images'
    batch = 16


    test_datagen = ImageDataGenerator(rescale = 1./255)

    test_generator = test_datagen.flow_from_directory(test_dir, 
                                                      target_size = (img_size,img_size),
                                                      batch_size = batch,
                                                       shuffle = False,
                                                       class_mode = 'categorical')
    
    return(test_generator)

In [25]:
def load_images(train, directory):
    # This function loads the images, resizes them and puts them into an array
    
    img_size = 448
    train_image = []
    for name in train['image_id']:
        path = directory + name + '.jpg'
        img = cv2.imread(path)
        img = cv2.resize(img, (img_size, img_size))
        train_image.append(img)
    train_image_array = np.array(train_image)
    
    return train_image_array

In [26]:
def load_generator_flow():
    # This function loads the generator for the test.
    # This second approach takes the test images from the memory
    
    from keras.preprocessing.image import ImageDataGenerator
    
    directory_data = 'C:/Users/julen/OneDrive/Escritorio/IA/CS577-Deep-Learning/Project/Data/'
    df_test = pd.read_csv(directory_data + 'test.csv')
    directory_images = 'C:/Users/julen/OneDrive/Escritorio/IA/CS577-Deep-Learning/Project/images/'
    x_test = load_images(df_test, directory_images)
    

    test_datagen = ImageDataGenerator(rescale = 1./255)


    test_generator = test_datagen.flow(x_test, batch_size = 4,
                                       #target_size = (224,224),
                                       shuffle = False)
                                       #classes = ['CROP'])
    
    return(df_test, test_generator)

In [27]:
from keras import backend as K
from keras.layers import Layer, InputSpec
from keras.legacy import interfaces

class GlobalKMaxPooling2D(Layer): #Inherits the properties of Layer class
    # Implements GlobalKMaxPooling, as it is needed for uploading the model
    
    def __init__(self, data_format=None, k = 10, **kwargs):
        super(GlobalKMaxPooling2D, self).__init__(**kwargs)
        self.data_format = K.normalize_data_format(data_format)
        self.input_spec = InputSpec(ndim=4)
        self.k = k

    def compute_output_shape(self, input_shape):
        if self.data_format == 'channels_last':
            return (input_shape[0], input_shape[3])
        else:
            return (input_shape[0], input_shape[1])

    def get_config(self):
        config = {'data_format': self.data_format, 'k' : self.k}
        base_config = super(GlobalKMaxPooling2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
    
        
    def call(self, inputs):
        if self.data_format == 'channels_last':
            k = self.k

            input_reshaped = tf.reshape(inputs, [tf.shape(inputs)[0], -1, tf.shape(inputs)[3]])
            input_reshaped = tf.reshape(input_reshaped, [tf.shape(input_reshaped)[0], tf.shape(input_reshaped)[2], tf.shape(input_reshaped)[1]])
            top_k = tf.math.top_k(input_reshaped, k=k, sorted = True, name = None)[0]
            mean = tf.keras.backend.mean(top_k, axis = 2)
            #assert ((input_reshaped.get_shape()[0], input_reshaped.get_shape()[-1]) == mean.get_shape())
        
        return mean

In [28]:
def load_model(name_json, name_h5):
    # This function loads the trained model
    
    json_file = open(name_json + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json, custom_objects = {'GlobalKMaxPooling2D': GlobalKMaxPooling2D})
    # load weights into new model
    loaded_model.load_weights(name_h5 + ".h5")
    print("Loaded model from disk")
    return loaded_model

In [29]:
def predict_and_save(model, test_generator, name):
    # This function does the prediction, modifies the test datafrane and saves it
    
    x_pred = model.predict_generator(test_generator, verbose = 1)
    df_test['healthy'] = x_pred[:,0]
    df_test['multiple_diseases'] = x_pred[:,1]
    df_test['rust'] = x_pred[:,2]
    df_test['scab'] = x_pred[:,3]
    df_test.to_csv(name, index = None)
    print('Prediction saved')

In [30]:
directory_load = 'C:/Users/julen/OneDrive/Escritorio/IA/CS577-Deep-Learning/Project/Models/'
name_load_h5 = 'Elope_xception_V2_best_model'
name_load_json = 'Elope_xception_V3_flow'

model = load_model(name_json = directory_load + name_load_json, name_h5 = directory_load + name_load_h5)

Loaded model from disk


In [31]:
df_test, test_generator = load_generator_flow()
predict_and_save(model, test_generator, name = 'Elope_xception_V3.csv')

456/456 [==============================] - 39s 86ms/step
[[8.31219740e-03 1.11466283e-02 9.80273604e-01 2.67632131e-04]
 [8.76260237e-05 3.83779668e-04 9.99526501e-01 2.20245738e-06]
 [1.97638190e-04 3.07090324e-03 3.49942384e-05 9.96696472e-01]
 ...
 [5.39765169e-04 2.39205151e-03 9.97014284e-01 5.37835440e-05]
 [9.86957788e-01 7.32123805e-03 8.77724553e-04 4.84323082e-03]
 [5.22935239e-04 8.09589773e-03 1.19286546e-04 9.91261959e-01]]
Prediction saved


In [32]:
df_test

,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,0.008312,0.011147,0.980274,0.000268
1,Test_1,0.000088,0.000384,0.999527,0.000002
2,Test_2,0.000198,0.003071,0.000035,0.996696
3,Test_3,0.998264,0.000166,0.000362,0.001208
4,Test_4,0.007931,0.018639,0.973199,0.000231
...,...,...,...,...,...
1816,Test_1816,0.001388,0.001787,0.996771,0.000054
1817,Test_1817,0.000887,0.003594,0.000048,0.995471
1818,Test_1818,0.000540,0.002392,0.997014,0.000054
1819,Test_1819,0.986958,0.007321,0.000878,0.004843


# Ensembling

In [66]:
import pandas as pd

directory = 'C:/Users/julen/OneDrive/Escritorio/IA/CS577-Deep-Learning/Project/Predictions/'
df_1 = pd.read_csv(directory + 'Best_score.csv')
df_2 = pd.read_csv(directory + 'Elope_probe_V1.csv')
df_3 = pd.read_csv(directory + 'Elope_xception_V3.csv')
df_total = df_1.copy()

In [67]:
df_total['healthy'] = (df_1['healthy'] + df_2['healthy'] + df_3['healthy']) / 3
df_total['multiple_diseases'] = (df_1['multiple_diseases'] + df_2['multiple_diseases'] + df_3['multiple_diseases']) / 3
df_total['rust'] = (df_1['rust'] + df_2['rust'] + df_3['rust']) / 3
df_total['scab'] = (df_1['scab'] + df_2['scab'] + df_3['scab']) / 3

In [68]:
df_total.to_csv('Ensembling.csv', index = None)

In [80]:
df_total.to_csv('Ensembling_optimized.csv', index = None)